In [71]:
import pandas as pd
import datetime
import MySQLdb
import os
import warnings
warnings.filterwarnings("ignore")

In [113]:
db.close()

In [77]:
dbname = 'cy_oi_data'
cursor = db.cursor()

name = '`云晨期货`'

sql = "ALTER TABLE " + dbname + ".v_rank ADD COLUMN " + name + " TEXT"
cursor.execute(sql)
sql = "ALTER TABLE " + dbname + ".b_rank ADD COLUMN " + name + " TEXT"
cursor.execute(sql)
sql = "ALTER TABLE " + dbname + ".s_rank ADD COLUMN " + name + " TEXT"
cursor.execute(sql)
sql = "ALTER TABLE " + dbname + ".v_increment ADD COLUMN " + name + " TEXT"
cursor.execute(sql)
sql = "ALTER TABLE " + dbname + ".b_increment ADD COLUMN " + name + " TEXT"
cursor.execute(sql)
sql = "ALTER TABLE " + dbname + ".s_increment ADD COLUMN " + name + " TEXT"
cursor.execute(sql)
sql = "ALTER TABLE " + dbname + ".v_volume ADD COLUMN " + name + " TEXT"
cursor.execute(sql)
sql = "ALTER TABLE " + dbname + ".b_oi ADD COLUMN " + name + " TEXT"
cursor.execute(sql)
sql = "ALTER TABLE " + dbname + ".s_oi ADD COLUMN " + name + " TEXT"
cursor.execute(sql)

294

In [82]:
dbname ='cy_oi_data'
cursor = db.cursor()

date = "'2018-5-25'"

sql = "delete from " + dbname + ".total_volume where Date >= " + date
cursor.execute(sql)

sql = "delete from " + dbname + ".v_rank where Date >= " + date
cursor.execute(sql)
sql = "delete from " + dbname + ".b_rank where Date >= " + date
cursor.execute(sql)
sql = "delete from " + dbname + ".s_rank where Date >= " + date
cursor.execute(sql)
sql = "delete from " + dbname + ".v_volume where Date >= " + date
cursor.execute(sql)
sql = "delete from " + dbname + ".b_oi where Date >= " + date
cursor.execute(sql)
sql = "delete from " + dbname + ".s_oi where Date >= " + date
cursor.execute(sql)
sql = "delete from " + dbname + ".v_increment where Date >= " + date
cursor.execute(sql)
sql = "delete from " + dbname + ".b_increment where Date >= " + date
cursor.execute(sql)
sql = "delete from " + dbname + ".s_increment where Date >= " + date
cursor.execute(sql)

0

In [79]:
db = MySQLdb.connect("localhost","root","123456")
db.set_character_set("gbk")

In [87]:
db = MySQLdb.connect("47.100.14.137","hyps44413","Nanhua")
db.set_character_set('gbk')

In [88]:
contract_list = ['ap','cf','cy','er','fg','jr','lr','ma','oi','pm','ri','rm','rs','sf','sm','sr','ta','wh','zc']
# contract_list = ['ap','cf']
FODER_DIR = "../各交易所期貨持倉量/鄭商所/"

for contract_id in contract_list:
    
    print(contract_id)
    cursor = db.cursor()
    
    DATA_DIR = FODER_DIR + contract_id
    
    dbname1 = contract_id + "_oi_data"
    dbname2 = contract_id + "_oi_original_data"
    
    company_order = pd.read_excel("../期貨公司名稱/鄭商所/" + contract_id + "_company_name.xlsx",header=None)
    
#     sql = "CREATE DATABASE IF NOT EXISTS " + dbname1
#     cursor.execute(sql)
#     sql = "CREATE DATABASE IF NOT EXISTS " + dbname2
#     cursor.execute(sql)
    
#     create_total_volume(dbname1)
    
#     V_info_create(dbname1,company_order)
#     B_info_create(dbname1,company_order)
#     S_info_create(dbname1,company_order)
    
    for filename in os.listdir(DATA_DIR):
        
        split_name = filename.split("_")
        getname = split_name[0]
        date = split_name[1].split(".csv")[0]
        
        date = datetime.datetime.strptime(date,'%Y%m%d')
    
        if (date >= datetime.datetime(2018,5,25)):
            
            df = pd.read_csv(DATA_DIR+"/"+filename,engine="python",encoding="utf-8-sig")
            df = df.where((pd.notnull(df)), None)
            date = date.strftime('%Y-%m-%d')
            
            for i in range(len(df)):
                for j in range(10):
                    if(str(df.loc[i][j]).strip() == "-"):
                        df.set_value(i, j , None, takeable=True)  #塞原始資料

                    elif((j==2 or j==3 or j==5 or j==6 or j==8 or j==9) and df.loc[i][j] != None):
                        df.set_value(i, j, str(df.iloc[i][j]).replace(',',''),takeable=True)


#             total_insert(df, filename.split(".csv")[0], dbname2, 0)

            V_info_insert(df, getname, date, dbname1)
            B_info_insert(df, getname, date, dbname1)
            S_info_insert(df, getname, date, dbname1)
            
            total_volume_insert(df, date, getname, dbname1)

ap
cf
cy
er
fg
jr
lr
ma
oi
pm
ri
rm
rs
sf
sm
sr
ta
wh
zc


In [74]:
b = []
c = []

a = df['4']
a = pd.DataFrame(list(a))

for i in range(len(a) - 1):
    c.append(a.loc[i][0])

for i in range(len(company_order)):
    b.append(company_order.loc[i][0])
    
for x in c:
    if x.strip() not in b:
        print(x)

会员简称      
云晨期货      


In [60]:
df

,0,1,2,3,4,5,6,7,8,9
0,名次,会员简称,成交量（手）,增减量,会员简称,持买仓量,增减量,会员简称,持卖仓量,增减量
1,1,华信期货,7237,7237,宝城期货,203,203,永安期货,965,965
2,2,海通期货,3460,3460,永安期货,191,191,华信期货,159,159
3,3,中信建投,3206,3206,中原期货,165,165,国投安信期货,151,151
4,4,国泰君安,2884,2884,华泰期货,163,163,国富期货,151,151
5,5,华安期货,2532,2532,广发期货,132,132,东兴期货,150,150
6,6,广州金控,2472,2472,恒泰期货,123,123,倍特期货,139,139
7,7,徽商期货,2445,2445,东航期货,118,118,华泰期货,135,135
8,8,宏源期货,2334,2334,华信期货,114,114,银河期货,123,123
9,9,永安期货,2332,2332,长江期货,114,114,南华期货,109,109


In [3]:
def total_insert(data,tbname,databaseName,start_index):       
    cursor = db.cursor()
    
    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + "." + tbname + "(\
    Rank INT(5),\
    Member CHAR(10),\
    Volume FLOAT(5),\
    Increment FLOAT(5),\
    Member1 CHAR(10),\
    Buy_Volume INT(10),\
    Increment_1 FLOAT(5),\
    Member2 CHAR(10),\
    Sales_Volume FLOAT(5),\
    Increment_2 FLOAT(5) )"

    cursor.execute(sql)
    value=[]

    for i in range(start_index,len(data)-1):      
        value.append((str(data.loc[i][0]).strip(),str(data.loc[i][1]).strip(),str(data.loc[i][2]).strip(),str(data.loc[i][3]).strip(),\
                      str(data.loc[i][4]).strip(),str(data.loc[i][5]).strip(),str(data.loc[i][6]).strip(),str(data.loc[i][7]).strip(),\
                      str(data.loc[i][8]).strip(),str(data.loc[i][9]).strip()))
    
    sql = "INSERT INTO " + databaseName + "." + tbname +" VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"        
    cursor.executemany(sql,value)
    db.commit()
    cursor.close()

In [4]:
def create_total_volume(databaseName):
    cursor = db.cursor()
    sql = "CREATE TABLE IF NOT EXISTS " + databaseName +".total_volume (Date DATE ,Contract_id TEXT(5), \
    V_Valume FLOAT,V_Increment FLOAT,\
    B_Valume FLOAT,B_Increment FLOAT,\
    S_Valume FLOAT,S_Increment FLOAT)"

    cursor.execute(sql)
    
def total_volume_insert(data, table_date, contract_id, databaseName):
    cursor = db.cursor()
    total_len = len(data)-1
    sql = "INSERT INTO " + databaseName + ".total_volume VALUES(%s,%s,%s,%s,%s,%s,%s,%s)"
    
    if(str(df.loc[total_len][1]) == 'None' or str(df.loc[total_len][1]).strip() == ""):          
        cursor.execute(sql,(table_date,contract_id,str(data.loc[total_len][2]).strip(),str(data.loc[total_len][3]).strip(),\
                            str(data.loc[total_len][5]).strip(),str(data.loc[total_len][6]).strip(),\
                            str(data.loc[total_len][8]).strip(),str(data.loc[total_len][9]).strip()))
    else:
        cursor.execute(sql,(table_date,contract_id,str(data.loc[total_len][1]).strip(),str(data.loc[total_len][2]).strip(),\
                            str(data.loc[total_len][4]).strip(),str(data.loc[total_len][5]).strip(),\
                            str(data.loc[total_len][7]).strip(),str(data.loc[total_len][8]).strip()))
        db.commit()
        cursor.close()

In [5]:
def V_info_create(databaseName, company_order):
    cursor = db.cursor()
    name=""
    for i in range(len(company_order)-1):
        name = name + company_order.loc[i][0] + ' TEXT,'
    name = name + company_order.loc[len(company_order)-1][0] + " TEXT"

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName +".v_rank ( Date DATE, Contract_id VARCHAR(50), " + name + ",\
           PRIMARY KEY(Date, Contract_id)) ENGINE=MyISAM"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".v_volume ( Date DATE , Contract_id VARCHAR(50), " + name + ",\
           PRIMARY KEY(Date, Contract_id)) ENGINE=MyISAM"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".v_increment ( Date DATE , Contract_id VARCHAR(50), " + name + ",\
           PRIMARY KEY(Date, Contract_id)) ENGINE=MyISAM"
    cursor.execute(sql)
    cursor.close()

def V_info_insert(data, contract_name, table_date, databaseName):
    cursor = db.cursor()
    
    empty = ""
    empty = empty + "'" + table_date + "', '" + contract_name + "'"


    sql = "INSERT INTO " + databaseName + ".v_rank (Date, Contract_id) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".v_volume (Date, Contract_id) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".v_increment (Date, Contract_id) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()
    
    for i in range(1,len(data)-1):
        if(data.loc[i][1] == None):
            continue
        
        else:
            sql = "UPDATE " + databaseName + ".v_rank SET " + str(data.loc[i][1]).strip() + "=" + str(data.loc[i][0]).strip() + \
                  " WHERE Date='" + table_date + "' and Contract_id = '" + contract_name + "'"
            cursor.execute(sql)
            db.commit()    

    for i in range(1,len(data)-1):
        if(data.loc[i][1] == None):
            continue
            
        else:
            sql = "UPDATE " + databaseName + ".v_volume SET " + str(data.loc[i][1]).strip() + "=" + str(data.loc[i][2]).strip() + \
                  " WHERE Date='" + table_date + "' and Contract_id = '" + contract_name + "'"
            cursor.execute(sql)
            db.commit()

    for i in range(1,len(data)-1):
        if(data.loc[i][1] == None):
            continue
            
        else:
            sql = "UPDATE " + databaseName + ".v_increment SET " + str(data.loc[i][1]).strip() + "=" + str(data.loc[i][3]).strip() + \
                 " WHERE Date='" + table_date + "' and Contract_id = '" + contract_name + "'"
            cursor.execute(sql)
            db.commit()
    
    cursor.close()

## B名次、買單量、增減

def B_info_create(databaseName, company_order):
    cursor = db.cursor()
    name=""
    for i in range(len(company_order)-1):
        name = name + company_order.loc[i][0] + ' TEXT,'
    name = name + company_order.loc[len(company_order)-1][0] + " TEXT"

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".b_rank ( Date DATE , Contract_id VARCHAR(50), " + name + ",\
           PRIMARY KEY(Date, Contract_id)) ENGINE=MyISAM"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".b_oi ( Date DATE , Contract_id VARCHAR(50), " + name + ",\
           PRIMARY KEY(Date, Contract_id)) ENGINE=MyISAM"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".b_increment ( Date DATE , Contract_id VARCHAR(50), " + name + ",\
           PRIMARY KEY(Date, Contract_id)) ENGINE=MyISAM"
    cursor.execute(sql)
    cursor.close()

def B_info_insert(data, contract_name, table_date, databaseName):
    cursor = db.cursor()
 
    empty = ""
    empty = empty + "'" + table_date + "', '" + contract_name + "'"
    
    sql = "INSERT INTO " + databaseName + ".b_rank (Date, Contract_id) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".b_oi (Date, Contract_id) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".b_increment (Date, Contract_id) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()
    
    for i in range(1,len(data)-1):
        if(data.loc[i][4] == None):
            continue  
        else:
            sql = "UPDATE " + databaseName + ".b_rank SET " + str(data.loc[i][4]).strip() + "=" + str(data.loc[i][0]).strip() + \
                 " WHERE Date='" + table_date + "' and Contract_id = '" + contract_name + "'"
            cursor.execute(sql)
            db.commit()    

    for i in range(1,len(data)-1):
        if(data.loc[i][4] == None):
            continue
            
        else:
            sql = "UPDATE " + databaseName + ".b_oi SET " + str(data.loc[i][4]).strip() + "=" + str(data.loc[i][5]).strip() + \
                 " WHERE Date='" + table_date + "' and Contract_id = '" + contract_name + "'"
            cursor.execute(sql)
            db.commit()

    for i in range(1,len(data)-1):
        if(data.loc[i][4] == None):
            continue
        
        else:
            sql = "UPDATE " + databaseName + ".b_increment SET " + str(data.loc[i][4]).strip() + "=" + str(data.loc[i][6]).strip() + \
                 " WHERE Date='" + table_date + "' and Contract_id = '" + contract_name + "'"
            cursor.execute(sql)
            db.commit()

    cursor.close()

## S名次、賣單量、增減

def S_info_create(databaseName, company_order):
    cursor = db.cursor()
    name=""
    
    for i in range(len(company_order)-1):
        name = name + company_order.loc[i][0] + ' TEXT,'
    name = name + company_order.loc[len(company_order)-1][0] + " TEXT"

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName +".s_rank ( Date DATE , Contract_id VARCHAR(50), " + name + ",\
           PRIMARY KEY(Date, Contract_id)) ENGINE=MyISAM"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".s_oi ( Date DATE , Contract_id VARCHAR(50), " + name + ",\
           PRIMARY KEY(Date, Contract_id)) ENGINE=MyISAM"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".s_increment ( Date DATE , Contract_id VARCHAR(50), " + name + ",\
           PRIMARY KEY(Date, Contract_id)) ENGINE=MyISAM"
    cursor.execute(sql)
    cursor.close()

def S_info_insert(data, contract_name, table_date, databaseName):
    cursor = db.cursor()

    empty = ""
    empty = empty + "'" + table_date + "', '" + contract_name + "'"
    
    sql = "INSERT INTO " + databaseName + ".s_rank (Date, Contract_id) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".s_oi (Date, Contract_id) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".s_increment (Date, Contract_id) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()
    
    for i in range(1,len(data)-1):
        if(data.loc[i][7] == None):
            continue  
        
        else:
            sql = "UPDATE " + databaseName + ".s_rank SET " + str(data.loc[i][7]).strip() + "=" + str(data.loc[i][0]).strip() + \
                 " WHERE Date='" + table_date + "' and Contract_id = '" + contract_name + "'"
            cursor.execute(sql)
            db.commit()    

    for i in range(1,len(data)-1):
        if(data.loc[i][7] == None):
            continue
            
        else:
            sql = "UPDATE " + databaseName + ".s_oi SET " + str(data.loc[i][7]).strip() + "=" + str(data.loc[i][8]).strip() + \
                 " WHERE Date='" + table_date + "' and Contract_id = '" + contract_name + "'"
            cursor.execute(sql)
            db.commit()

    for i in range(1,len(data)-1):
        if(data.loc[i][7] == None):
            continue
            
        else:
            sql = "UPDATE " + databaseName +".s_increment SET " + str(data.loc[i][7]).strip() + "=" + str(data.loc[i][9]).strip() + \
                 " WHERE Date='" + table_date + "' and Contract_id = '" + contract_name + "'"
            cursor.execute(sql)
            db.commit()

    cursor.close()

In [109]:
db = MySQLdb.connect("localhost","root","123456","er_oi_original_data")
db.set_character_set('gbk')

In [110]:
cursor = db.cursor()
sql = 'show tables'
cursor.execute(sql)
data = cursor.fetchall()

In [111]:
company_list = []
for table in data:
    sql = 'select Member,Member1,Member2 from ' + table[0]
    cursor.execute(sql)
    name = cursor.fetchall()   
    for i in name:
        for j in range(3):
            if (i[j] not in company_list and i[j]!=None):
                company_list.append(i[j])

In [112]:
a = pd.DataFrame(company_list)
a.to_excel("../期貨公司名稱/er_company_name.xlsx",index=None,header=None)

In [ ]:
df = pd.read_excel("../期貨公司名稱/鄭商所/123.xlsx",sheet_name=1)

In [ ]:
df